# Installing the necessary packages

In [5]:
!pip install -q streamlit pyngrok pydub google-cloud-translate langdetect transformers beautifulsoup4 requests
!pip install -q streamlit-webrtc torch datasets
!pip install -U bitsandbytes
!pip install -U accelerate
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft


ERROR: Could not find a version that satisfies the requirement bitsandbytes (from versions: none)
ERROR: No matching distribution found for bitsandbytes


  Cloning https://github.com/unslothai/unsloth.git to c:\users\grise\appdata\local\temp\pip-install-zhddmzdo\unsloth_3b487cbccd67489e85bfe0a2a93d8846


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?
ERROR: Could not find a version that satisfies the requirement xformers<0.0.27 (from versions: none)
ERROR: No matching distribution found for xformers<0.0.27


# Data Scraping and Preprocessing

In [1]:
%%capture

from transformers import AutoTokenizer, AutoModelForCausalLM
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

def scrape_data():
    urls = [
        'https://devinterview.io/questions/machine-learning-and-data-science/data-processing-interview-questions',
        'https://www.themuse.com/advice/interview-questions-and-answers',
        'https://novoresume.com/career-blog/interview-questions-and-best-answers-guide',
        'https://www.glassdoor.com/Interview/nursing-interview-questions-SRCH_KO0,7.htm',
        'https://www.datainterview.com/blog/100-data-science-interview-questions',
        'https://interviewprep.org/finance-interview-questions/'
    ]

    with open('interview_questions.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Question', 'Answer'])

        for url in urls:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            if 'glassdoor' in url:
                questions = soup.find_all('span', class_='questionText')
                answers = soup.find_all('span', class_='answerText')
            else:
                questions = soup.find_all('h3')
                answers = soup.find_all('p')

            for question, answer in zip(questions, answers):
                question_text = question.get_text(strip=True)
                answer_text = answer.get_text(strip=True)
                writer.writerow([question_text, answer_text])

scrape_data()

# Load and clean dataset
df = pd.read_csv('interview_questions.csv')
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
print(f"Cleaned Dataset size: {len(df)} samples")

#function to generate questions
def generate_questions(field, model, tokenizer):
    prompt = f"Generate five interview questions for a candidate applying for a position in {field}."
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    questions = generated_text.split('\n')
    return questions[:5] if len(questions) >= 5 else questions



ReadTimeout: HTTPSConnectionPool(host='devinterview.io', port=443): Read timed out. (read timeout=None)

# Load and Fine-Tune the Model

In [2]:
from unsloth import FastLanguageModel
from datasets import Dataset

data = {
    "input_text": df['Question'].astype(str).tolist(),
    "output_text": df['Answer'].astype(str).tolist()
}
dataset = Dataset.from_dict(data)
print(f"Dataset size: {len(dataset)} samples")

max_seq_length = 512
model_name = "unsloth/llama-3-8b-bnb-4bit"

print("Loading model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)
print("Model loaded successfully.")

print("Applying PEFT configuration...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)
print("PEFT configuration applied.")

model.eval()
print("Model set to evaluation mode.")




ModuleNotFoundError: No module named 'unsloth'

# saving model and tokenizer

In [6]:
save_directory = "./saved_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/tokenizer.json')

# Generate and Evaluate Responses

In [ ]:
import torch
from difflib import SequenceMatcher

def evaluate_user_answer(generated_question, user_answer, correct_answer):
    def similarity(a, b):
        return SequenceMatcher(None, a, b).ratio()
    score = similarity(user_answer.lower(), correct_answer.lower())
    evaluation = "Correct" if score > 0.8 else "Incorrect"
    return evaluation, score

# Sample inputs for testing
sample_inputs = [
    "What are the key challenges in data preprocessing?",
    "How do you handle missing data in datasets?",
    "Explain the concept of feature scaling.",
    "What are your strengths and weaknesses?",
    "Describe a time you faced a challenge at work."
]

correct_answers = [
    "Data preprocessing challenges include missing data, outliers, and noisy data.",
    "Handling missing data involves methods like imputation, deletion, or using algorithms that support missing values.",
    "Feature scaling is the process of normalizing the range of independent variables or features.",
    "My strengths include diligence and problem-solving; my weakness is perfectionism.",
    "I faced a challenge with a tight deadline at work and overcame it by prioritizing tasks and effective time management."
]

print("Starting evaluation...")
for idx, input_text in enumerate(sample_inputs):
    print(f"\nProcessing input {idx + 1}/{len(sample_inputs)}")
    print(f"Input: {input_text}")

    user_answer = input("Your Answer: ")
    evaluation, score = evaluate_user_answer(input_text, user_answer, correct_answers[idx])
    print(f"Evaluation: {evaluation} (Score: {score})\n")

print("Evaluation completed.")


Starting evaluation...

Processing input 1/5
Input: What are the key challenges in data preprocessing?
